## Analyse bacteria colony sizes

**You can solve this exercise in pairs and/or discuss with your collegues as much as you like**

The directory ```data/bacteria/``` contains "microscope images" from experiments done with three different bacteria, b1, b2 and b3, each treated with three different drugs, d1, d2 and d3 plus control group for each bacteria.
The image are named according to bacteria and drug.
We will process and analyse the images to see how the drugs have affected the sizes of the colonies the bacterias create. We will look at the area of the colonies.

Then we will create a data table with all the collected data.

Look at the images and see if you can figure out what preprocessing need to be done before labeling the colonies.

Basic algorithmic layout:

- import needed libraries (start with the ones you know and add more as you go along)

- use a for loop to loop over the needed files in the directory and to the following for each file:
    you probably need the index as well

- open the tiff image and fetch the user defined metadata and read the image data.

- preprocess
- apply filters

- remove items touching border

- connected component analysis
labeling

- print how many colonies are found
- plot the image

- Store the loaded images,the labeld images and the metadata in arrays for later use!

In [ ]:

#Write your solution here

Now, lets collect the data and store it in a data fram for statistical analysis later

Create tables of regions of all the loaded images and labled images by using '''skimage.measure.regionprops_table(...)'''

Create panda frames from each table by adding the bacteria, the drug and the area for each colony.
Bacteria and Drug are stored int he metadata.

Use json package to read out bacteria and drug for each image/metadata

Concatenate all frames to a single frame and save it as csv


Now we will gather the data we are interested in from the labeled images and store it in a table.
We will also add information to the the tables to identify the bacteria and drug used.

Algorithm outline:

- import the needed libraries as before

- crete a dataframe to store all other frames

- loop over all the labeled image (the array you created)

- crete the properties table using ```skimage.measure.regionprops_table( labeld image, original image, properties=['area','feret_diameter_max'])```

- create the dataframe from the properties table

- read the bacteria and drug metadata from the array before

- add the columns 'bacteria' and 'drug' to the dataframe and set the respective data in it

- add the frame to the toal frame

- after the for loop, save the total frame and print some information about the total frame

In [ ]:

#Write your solution here